In [543]:
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [468]:
need_cols = ["cowcode","year", "ager", "agexp", "anoc", "army85", 
           "autch98", "auto4", "autonomy", "avgnabo",
           "centpol3", "coldwar", "decade1", "decade2", 
           "decade3", "decade4", "dem", "dem4",
           "demch98", "dlang", "drace", "drel", 
           "durable", "ef", "ef2", "ehet", 
           "elfo", "elfo2", "etdo4590", "expgdp", 
           "exrec", "fedpol3", "fuelexp", "gdpgrowth", 
           "geo1", "geo2", "geo34", "geo57",
           "geo69", "geo8", "illiteracy", "incumb", 
           "infant", "inst", "inst3", "life",
           "lmtnest", "major", "manuexp", "milper",
           "mirps0", "mirps1", "mirps2", "mirps3", 
           "nat_war", "ncontig", "nmgdp", "nmdp4_alt",
           "numlang", "nwstate", "oil", "p4mchg", 
           "parcomp", "parreg", "part", "partfree", 
           "plural", "plurrel", "pol4", "pol4m", 
           "pol4sq", "polch98", "polcomp", "popdense", 
           "presi", "pri", "proxregc", "reg", 
           "regd4_alt", "relfrac", "seceduc", "second",
           "semipol3", "sip2", "sxpnew", "sxpsq",
           "tnatwar", "trade", "warhist", "xconst", 'warstds']

In [469]:
seed = 20201106 # the deadline date

In [470]:
# Load raw data 
df = pd.read_csv('data/SambnisImp.csv', usecols = need_cols )

# Print dimension information about the dataset
print(f'Raw data dimensions: {df.shape}')

# Print first 5 columns
df.head()

Raw data dimensions: (7140, 91)


,cowcode,year,warstds,autonomy,popdense,army85,milper,trade,nmgdp,autch98,...,mirps1,mirps2,mirps3,sxpsq,pol4sq,decade1,decade2,decade3,decade4,proxregc
0,700,1945,0,0.005151,118.554791,129472.9042,121.087366,72.881375,4508.131692,0,...,0.16935,0.313143,0.373714,0.052989,61.493014,0,0,0,0,0.143299
1,700,1946,0,0.000000,117.756342,129413.0225,121.885359,72.900089,4491.340308,0,...,0.00000,1.000000,0.000000,0.052663,100.000000,0,0,0,0,1.000000
2,700,1947,0,0.000000,118.280656,130431.0145,122.780608,72.962880,4484.267686,0,...,0.00000,1.000000,0.000000,0.052891,100.000000,0,0,0,0,1.000000
3,700,1948,0,0.000000,118.325869,126781.6866,118.256427,73.102449,4474.082672,0,...,0.00000,1.000000,0.000000,0.052902,100.000000,0,0,0,0,1.000000
4,700,1949,0,0.000000,118.312296,130979.2470,122.245074,72.850389,4497.299503,0,...,0.00000,1.000000,0.000000,0.052706,100.000000,0,0,0,0,1.000000


In [471]:
df["year"] = pd.to_datetime(df["year"], format="%Y")
df = df.set_index(['cowcode', 'year'])

In [547]:
serial_len = 10
X = []
y = []
for name, group in df.groupby(level='cowcode'):
    group = group.reset_index(level='cowcode', drop=True)
    for i in range(len(group)-serial_len-1):
        y.append((group.iloc[i+serial_len+1,:])["warstds"])
        X.append(group.iloc[i:i+serial_len].drop("warstds", axis=1).values)
print(np.shape(X), np.shape(y))

(5219, 10, 88) (5219,)


In [548]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

In [549]:
print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))

(3653, 10, 88) (1566, 10, 88) (3653,) (1566,)


In [550]:
scalers = {}
X_trainsc = np.zeros_like(X_train)
X_testsc = np.zeros_like(X_test)
for i in range(X_train.shape[1]):
    scalers[i] = MinMaxScaler()
    X_trainsc[:, i, :] = scalers[i].fit_transform(X_train[:, i, :]) 

for i in range(X_test.shape[1]):
    X_testsc[:, i, :] = scalers[i].transform(X_test[:, i, :]) 

In [551]:
print(np.unique(y_train,return_counts=True))

(array([0., 1.]), array([3611,   42], dtype=int64))


In [ ]:
weights = {0:1,1:50}

# design network
model = Sequential()
model.add(LSTM(20, input_shape=(X_trainsc.shape[1], X_trainsc.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
print(model.summary())
# fit network
history = model.fit(X_trainsc, y_train, epochs=30, batch_size=1000, validation_data=(X_testsc, y_test), verbose=2, shuffle=False, class_weight=weights)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 20)                8720      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 21        
Total params: 8,741
Trainable params: 8,741
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
4/4 - 1s - loss: 1.1263 - val_loss: 0.5888
Epoch 2/30
4/4 - 0s - loss: 1.0431 - val_loss: 0.4779
Epoch 3/30
4/4 - 0s - loss: 0.9967 - val_loss: 0.4204
Epoch 4/30
4/4 - 0s - loss: 0.9663 - val_loss: 0.3993
Epoch 5/30
4/4 - 0s - loss: 0.9404 - val_loss: 0.4014
Epoch 6/30
4/4 - 0s - loss: 0.9166 - val_loss: 0.4153
Epoch 7/30


In [ ]:
y_pred = model.predict_classes(X_testsc)
print(np.unique(y_pred))

In [ ]:
confusion_matrix(y_test, y_pred)